In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

In [ ]:
#===========PREFACE============

#SIMPLE IMPLERMENTATION OF LINEAR REGRESSION USING SKLEARN. (MY **HELLO WORLD!** PROJECT IN THE WORLD OF ML :3)

#DATASET: some used cars with mileage, price, model , color features

#I`m approaching it by using a linear regression. My task is to predict Price column using the year,model,mileage,color,transmission columns

#=======ENDING OF PREFACE========

In [ ]:
import pandas as pd
import matplotlib
from urllib.request import urlretrieve
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

In [ ]:
#INSTALL DATA
data_df_link = 'https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/refs/heads/master/usedcars.csv'
urlretrieve(data_df_link, 'snsdata.csv')
cars_df = pd.read_csv('snsdata.csv')


In [ ]:
#LOSS FUNCTION
#I`m using RMSE (Root mean square error)

def rmse(targets, predictions):
    return np.sqrt(np.mean(np.square(targets-predictions)))

In [ ]:
#PRICE TO MILEAGE 

inputs, targets = cars_df[['mileage']], cars_df.price

model = LinearRegression()
model.fit(inputs, targets)

predictions = model.predict(inputs)

print(rmse(targets,predictions))

plt.plot(inputs, predictions, 'r', alpha = 0.9 )
plt.scatter(inputs, targets, s = 8, alpha=0.8 )
plt.xlabel("Mileage")
plt.ylabel("Price")
plt.legend(['Estimated', 'Actual'])
plt.show()

In [ ]:
#PRICE TO YEAR 

inputs, targets = cars_df[['year']], cars_df.price

model = LinearRegression()
model.fit(inputs, targets)

predictions = model.predict(inputs)

print(rmse(targets,predictions))

plt.plot(inputs, predictions, 'r', alpha = 0.9 )
plt.scatter(inputs, targets, s = 8, alpha=0.8 )
plt.xlabel("Mileage")
plt.ylabel("Price")
plt.legend(['Estimated', 'Actual'])
plt.show()

In [ ]:
#HANDLING CATEGORICAL PARAMETERS

#TRANSMISSION
transmission_codes = {'AUTO': 1, 'MANUAL':0}
cars_df['transmission_code'] = cars_df.transmission.map(transmission_codes)
print(cars_df.price.corr(cars_df.transmission_code))

#ONE-HOT ENCODING
#COLOR 
colors = set()
for i in cars_df.color:
    colors.add(i)

enc = preprocessing.OneHotEncoder()
enc.fit(cars_df[['color']])
one_hot = enc.transform(cars_df[['color']]).toarray()
cars_df[['Silver', 'Yellow', 'Red', 'Gold', 'Gray', 'White', 'Black', 'Green', 'Blue']] = one_hot
#COLOR
models = set()
for i in cars_df.model:
    models.add(i)

enc = preprocessing.OneHotEncoder()
enc.fit(cars_df[['model']])
one_hot = enc.transform(cars_df[['model']]).toarray()
cars_df[['SES', 'SE', 'SEL']] = one_hot


print(cars_df.head())

In [ ]:
#PREDICTING THE PRICE TO ALL PARAMETERS

input_cols = ['year', 'mileage', 'transmission_code', 'Silver', 'Yellow', 'Red', 'Gold', 'Gray', 'White', 'Black', 'Green', 'Blue', 'SES', 'SE', 'SEL']

inputs, targets = cars_df[input_cols], cars_df.price

model = LinearRegression().fit(inputs, targets)

predictions = model.predict(inputs)

print(rmse(targets, predictions))
plt.scatter(targets, cars_df.mileage, s=5, color='red', label='Targets')
plt.scatter(predictions, cars_df.mileage, s=5, color='blue', label='Predictions')
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.legend()
plt.show()

In [ ]:
#FEATURE SCALING

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

numeric_cols = ['mileage','year']
scaler = StandardScaler()
scaler.fit(cars_df[numeric_cols])

scaled_inputs = scaler.transform(cars_df[numeric_cols])

cat_cols = ['transmission_code', 'Silver', 'Yellow', 'Red', 'Gold', 'Gray', 'White', 'Black', 'Green', 'Blue', 'SES', 'SE', 'SEL']
categorical_data = cars_df[cat_cols].values


inputs = np.concatenate((scaled_inputs, categorical_data), axis=1)
targets = cars_df.price

model = Ridge(alpha=1.0).fit(inputs, targets)

predictions = model.predict(inputs)

print("RMSE AFTER FEATURE SCALING: ", rmse(predictions, targets))

weights_df = pd.DataFrame({
    'feature': np.append(numeric_cols + cat_cols, 'intercept'),
    'weight': np.append(model.coef_, model.intercept_)
})
weights_df = weights_df.sort_values('weight', ascending=False)

print(weights_df)
plt.figure(figsize=(10,6))
plt.barh(weights_df['feature'], weights_df['weight'])
plt.xlabel("Weight coef")
plt.ylabel("Feature")
plt.title("Feature weight")
plt.gca().invert_yaxis()  
plt.show()

In [ ]:
#TRYING TO PREDICT NEW CAR PRICE

new_car = {
    'mileage': 50000,
    'year': 2015,
    'transmission_code': 1,
    'Silver': 0,
    'Yellow': 0,
    'Red': 0,
    'Gold': 0,
    'Gray': 0,
    'White': 0,
    'Black': 1, 
    'Green': 0,
    'Blue': 0,
    'SES': 0,
    'SE': 1,     
    'SEL': 0
}

new_car_df = pd.DataFrame([new_car])

scaled_numeric = scaler.transform(new_car_df[['mileage', 'year']])

categorical_data = new_car_df[['transmission_code','Silver','Yellow','Red','Gold','Gray',
                               'White','Black','Green','Blue','SES','SE','SEL']].values

new_inputs = np.concatenate((scaled_numeric, categorical_data), axis=1)

predicted_price = model.predict(new_inputs)
print("Predicted price:", predicted_price[0])